<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/MicroGradEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from timeit import default_timer as timer
t1 = timer()
try:
  from google.colab import drive
  drive.mount("/content/drive/", force_remount = True)
  import torch, random, math
  import matplotlib.pyplot as plt
  import numpy as np
  print(f">>>> You are on CoLaB with torch version: {torch.__version__}")
except Exception as e:
  print(f">>>>{type(e)}: {e}\n>>>> Please correct {type(e)} and reload")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f">>>> Available device: {device}")
def mytimer(t: float = timer())->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h}, min: {m:>02}, sec: {s:>05.2f}"
print(f">>>> Time elapsed: \t{mytimer(timer() - t1)}")

Mounted at /content/drive/
>>>> You are on CoLaB with torch version: 2.0.1+cu118
>>>> Available device: cpu
>>>> Time elapsed: 	hrs: 0, min: 00, sec: 32.00


In [46]:
class Micro:

  def __init__(self,
               data,
               _children = (),
               _op = " ",
               label = " "):
    self.grad = 0.0
    self._backward = lambda: None
    self.data = data
    self._children = set(_children)
    self._op = _op
    self.label = label

  def __add__(self, other):
    other = other if isinstance(other, Micro) else Micro(other)
    out = Micro(self.data + other.data, (self, other), "+")

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    return out
  
  def __mul__(self, other):
    other = other if isinstance(other, Micro) else Micro(other)
    out = Micro(self.data * other.data, (self, other), "*")

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (float, int))
    out = Micro(self.data ** other, (self,), "pow")

    def _backward():
      self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward
    return out
  
  def tanh(self):
    x = self.data
    exp = math.exp(2 * x)
    t = (exp - 1) / (exp + 1)
    out = Micro(t, (self,), "tanh")
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    return out
  
  def sigmoid(self):
    x = self.data
    exp = math.exp(x)
    s = exp / (1 + exp)
    out = Micro(s, (self,), "sigmoid")

    def _backward():
      self.grad += s * (1 - s) * out.grad
    out._backward = _backward
    return out
  
  def relu(self):
    x = self.data
    r = 0 if x < 0 else x
    out = Micro(r, (self,), "ReLU")

    def _backward():
      self.grad += (r > 0) * out.grad
    out._backward = _backward
    return out
  
  def backward(self):
    topo = []
    visited = set()
    def build_topology(v):
      if v not in visited:
        visited.add(v)
        for child in v._children:
          build_topology(child)
        topo.append(v)
    build_topology(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()
  
  def __radd__(self, other):
    return self + other
  
  def __neg__(self):
    return self * (-1)
  
  def __rmul__(self, other):
    return  self * other
  
  def __sub__(self, other):
    return self + (-other)
  
  def __rsub__(self, other):
    return other + (-self)
  
  def __truediv__(self, other):
    return self * (other **-1)
  
  def __rtruediv__(self, other):
    return other * (self **-1)
  
  def __repr__(self):
    return f"Micro(data = {self.data}, grad = {self.grad})"




In [47]:
x1, x2 = Micro(2.0, label = "x1"), Micro(0.0, label = "x1")
w1, w2 = Micro(-3.0, label = "w1"), Micro(4.0, label = "w2")
x1w1 = w1 * x1 ; x1w1.label = "x1w1"
x2w2 = w2 * x2 ; x2w2.label = "x2w2"
x1w1x2w2 = x1w1 + x2w2 ; x1w1x2w2.label = "x1w1x2w2"
b = Micro(6.8917736, label = "bias")
n = x1w1x2w2 + b ; n.label = "neuron"
o = n.tanh() ; o.label = "output"
o.backward()

In [48]:
o.grad

1.0

In [49]:
assert n.grad == x1w1x2w2.grad == x1w1.grad == x2w2.grad 

In [50]:
assert w2.grad == 0.0

In [51]:
class Module:

  def zero_grad(self):
    for p in self.parameters():
      p.grad = 0
  
  def parameters(self):
    return []


In [52]:
class Neuron(Module):
  def __init__(self, nin, non_lin = True):
    self.w = [Micro(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Micro(random.uniform(-1,1))
    self.non_lin = non_lin

  def __call__(self, x):
    out = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    return out.relu() if self.non_lin else out
  
  def parameters(self):
    return [self.b] + self.w
  
  def __repr__(self):
    return f"{'ReLU' if self.non_lin else 'Linear'}Neuron({len(self.w)})"



In [53]:
class Layer(Module):
  def __init__(self, nin, n_out, **kwargs):
    self.neurons = [Neuron(nin, **kwargs) for _ in range(n_out)]
  
  def __call__(self, x):
    out = [n(x) for n in self.neurons]
    return out[0] if len(out) == 1 else out
  
  def parameters(self):
    return [p for n in self.neurons for p in n.parameters()]
  
  def __repr__(self):
    return f"Layer of[{', '.join(str(n) for n in self.neurons)}]"
  

In [54]:
class MLP(Module):
  def __init__(self, nin, n_out):
    l_total = [nin] + n_out
    self.layers = [Layer(l_total[i], l_total[i+1], non_lin = i != (len(n_out) - 1)) for i in range(len(n_out))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for l in self.layers for p in l.parameters()]
  
  def __repr__(self):
    return f"MLP of [{', '.join(str(l) for l in self.layers)}]"

In [85]:
input_dim = 4
inputs = [[2.0, 1.2, 3.1, 4.8],
          [0.3, 0.8, 1.2, 0.8],
          [2.9, 3.1, 4.0, 1.8],
          [0.2, 1.3, 1.1, 1.4],
          [3.1, 2.9, 3.4, 2.5]]
targets = [1, -1, 1, -1, 1]

n_out = [6, 8, 4, 1]

model = MLP(input_dim, n_out)

In [86]:
preds = [model(x) for x in inputs]

In [87]:
preds

[Micro(data = -0.17896520509257785, grad = 0.0),
 Micro(data = -0.14181040660046146, grad = 0.0),
 Micro(data = -0.2898397306240814, grad = 0.0),
 Micro(data = -0.17896520509257785, grad = 0.0),
 Micro(data = -0.17896520509257785, grad = 0.0)]

In [88]:
#[yi - pi for yi, pi in zip(targets, preds)]

In [95]:
# training loop
for epoch in range(10):
  preds = [model(x) for x in inputs]
  loss = sum((yi - pi)**2 for yi, pi in zip(targets, preds))
  model.zero_grad()
  loss.backward()
  for p in model.parameters():
    p.data += -0.01 * p.grad
  
  print(epoch, loss)

  

0 Micro(data = 0.20670489492110714, grad = 1.0)
1 Micro(data = 0.2033048744659951, grad = 1.0)
2 Micro(data = 0.15451289717291228, grad = 1.0)
3 Micro(data = 0.19032124529359765, grad = 1.0)
4 Micro(data = 0.2854279470244463, grad = 1.0)
5 Micro(data = 0.29022781518606416, grad = 1.0)
6 Micro(data = 0.19796330336247864, grad = 1.0)
7 Micro(data = 0.19697178421946543, grad = 1.0)
8 Micro(data = 0.14737930177339467, grad = 1.0)
9 Micro(data = 0.18011713304748425, grad = 1.0)
